# Classificador utilizando Redes Neurais Convolucionais
***

## Instalação dos pacotes
***

In [2]:
# pacote de inteligência artificial
import tensorflow as tf
# pacote de gerenciamento de conjuntos de dados
import pandas as pd

## Organizando o Dataframe
***

In [19]:
# conjunto de dados de treinamento, validação e teste
train_dataframe = pd.read_csv('../csv-data/braille_character1.csv', delimiter = ',', 
                              header = 0, usecols = ['Path', 'Labels'])
validation_dataframe = pd.read_csv('../csv-data/braille_character2.csv', delimiter = ',',
                                   header = 0, usecols = ['Path', 'Labels'])
test_dataframe = pd.read_csv('../csv-data/braille_character3.csv', delimiter = ',',
                             header = 0, usecols = ['Path', 'Labels'])

In [23]:
train_files = train_dataframe['Path']

for i in range(0, len(train_files)):
    train_files[i] = '../files/'

'image1-1.tiff'

In [17]:
len(validation_dataframe)

1326

In [18]:
len(train_dataframe)

1326